In [1]:
import pandas as pd
import os

DATA_DIR = 'data/'
POPULATION_DATA_DIR = DATA_DIR + 'population/'
PROCESSED_DATA_DIR = DATA_DIR + "processed/"

# Analyse du biais 

In [3]:
df = pd.read_csv(PROCESSED_DATA_DIR+'manual_transcription.csv')
df

,annee,xls_id,chef_vocation,chef_vocation_transcripted,chef_vocation_certitude,pensionnaires_condition,pensionnaires_condition_transcripted,pensionnaires_certitude,enfants_professions,enfants_profession_transcripted,enfants_profession_certitude
0,1848,1,froureux,NaN,5.0,·,NaN,NaN,NaN,NaN,NaN
1,1848,2,conservateur,conservateur,5.0,pensionnaire,pensionnaire,3.0,NaN,NaN,NaN
2,1848,3,boulanger,boulanger,5.0,·,NaN,NaN,NaN,NaN,NaN
3,1848,4,tailleuse,tailleuse,5.0,ouvriere|apprentie,ouvriere|apprentie,NaN,NaN,NaN,NaN
4,1848,5,relieur,relieur,5.0,·,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
518,1868,245,md de lamis,marchand de tamis,3.0,·,NaN,NaN,NaN,NaN,NaN
519,1868,246,rentiere,rentiere,5.0,arpenteur dque,arpenteur,5.0,NaN,NaN,NaN
520,1868,247,charcutier,charcutier,5.0,·,domestique,5.0,NaN,NaN,NaN
521,1868,248,macon,macon,5.0,pensionnaire,pensionnaire,5.0,NaN,NaN,NaN


In [7]:
df_1848 = df[df["annee"]==1848][["chef_vocation", "chef_vocation_transcripted", "pensionnaires_condition", "pensionnaires_condition_transcripted"]]
df_1868 = df[df["annee"]==1868][["chef_vocation", "chef_vocation_transcripted", "pensionnaires_condition", "pensionnaires_condition_transcripted"]]



In [10]:
df_1848

,chef_vocation,chef_vocation_transcripted,pensionnaires_condition,pensionnaires_condition_transcripted
0,froureux,NaN,·,NaN
1,conservateur,conservateur,pensionnaire,pensionnaire
2,boulanger,boulanger,·,NaN
3,tailleuse,tailleuse,ouvriere|apprentie,ouvriere|apprentie
4,relieur,relieur,·,NaN
...,...,...,...,...
269,·,instituteur,dque dque,pensionnaire|domestique
270,ptrepreneur|rontr,entrepreneur,·|pensre,domestique
271,NaN,rentiere,id dque|·|·,pensionnaire|pensionnaire|domestique
272,NaN,rentiere,jnsti pensre,domestique


In [12]:
man_corr = pd.read_csv(DATA_DIR+'/corrected.csv')
man_corr = man_corr[man_corr["corrected"]!= "NSP"]
man_corr = man_corr.dropna()
man_corr

,original,corrected
0,agent de police,agent de police
1,agricult,agriculteur
2,agriculteur id,agriculteur|idem
3,agt,agent
4,aide,aide
...,...,...
472,volontaire,volontaire
473,vve de chambre,valet de chambre
474,murier,armurier
475,griculteur,agriculteur


In [16]:
correction_mapping = dict(zip(man_corr['original'], man_corr['corrected']))

def apply_correction(term):
    term = correction_mapping.get(term, term)
    temp = [t for t in term.split("|")]
    return [t for t in temp if t!="NA" and t!="NSP"]

correction_mapping

{'agent de police': 'agent de police',
 'agricult': 'agriculteur',
 'agriculteur id': 'agriculteur|idem',
 'agt': 'agent',
 'aide': 'aide',
 'ailleuse': 'tailleuse',
 'aopriculteur': 'agriculteur',
 'app': 'apprenti',
 'app couturiere': 'apprentie couturiere',
 'app negociant': 'apprenti negociant',
 'app tailleuse': 'apprentie tailleuse',
 'apprentie modiste': 'apprentie modiste',
 'apprentier': 'apprenti',
 'apprentis': 'apprenti',
 'apprentisse': 'apprenti',
 'arpentier': 'charpentier',
 'aypographe': 'typographe',
 'bauquier': 'banquier',
 'beulanger': 'boulanger',
 'bijoulier': 'bijoutier',
 'bingere': 'lingere',
 'blanchi ause': 'blanchisseuse',
 'bonne': 'bonne',
 'cafelier': 'cafetier',
 'caiffeur': 'coiffeur',
 'catelier': 'cafetier',
 'chapellier': 'chapellier',
 'charon': 'charron',
 'charpent': 'charpentier',
 'charrelier': 'charrelier',
 'chirurgien': 'chirurgien',
 'cimenteur': 'cimenteur',
 'cinturier': 'ceinturier',
 'clerc': 'clerc',
 'cleve': 'eleve',
 'clever caveug 

In [14]:
def fillIdem(l):
    '''
    Return a list with "id" replaced by the previous term
    '''
    return_list = []
    prev = "idem"
    for term in l:
        if term != "idem":
            prev = term
        return_list.append(prev)
    return return_list

In [19]:
df_1848.fillna("")

df_1848["chef_vocation"] = df_1848["chef_vocation"].apply(lambda l: apply_correction(l))
df_1848["pensionnaires_condition"] = df_1848["pensionnaires_condition"].apply(lambda l: apply_correction(l))

df_1848["chef_vocation"] = df_1848["chef_vocation"].apply(lambda l: fillIdem(l))
df_1848["pensionnaires_condition"] = df_1848["pensionnaires_condition"].apply(lambda l: fillIdem(l))

AttributeError: 'float' object has no attribute 'split'